In [7]:
import psycopg2
import os
import pandas as pd
import json

In [8]:
postgres_usr = os.environ.get('POSTGRES_USR')
postgres_pwd = os.environ.get('POSTGRES_PWD')
postgres_host = os.environ.get('POSTGRES_HOST')

conn_uri = f'postgresql://{postgres_usr}:{postgres_pwd}@{postgres_host}:5432/postgres'

In [9]:
# Connect to your PostgreSQL database using cursor
# Execute DDL
def connect_database_ddl(exec_query:str, db, usr, pwd, host):
    conn = psycopg2.connect(f"dbname={db} user={usr} password={pwd} host={host}")
    cursor = conn.cursor()
    # Execute the TRUNCATE statement
    cursor.execute(exec_query)
    # Commit the changes
    conn.commit()
    # Close the cursor and connection
    cursor.close()
    conn.close()

In [28]:
t_names = ['orders', 'categories', 'customers','departments', 'order_items', 'products']

for t_name in t_names:
    exec_query = f'TRUNCATE TABLE retail.{t_name}'
    connect_database_ddl(exec_query, "postgres", postgres_usr, postgres_pwd, postgres_host)

print('Truncation Complete !')

Truncation Complete !


In [26]:
"""
Pandas read_sql can not execute DDL
"""

query = 'SELECT * FROM retail.customers'
pd.read_sql(query, conn_uri)

,customer_id,customer_fname,customer_lname,customer_email,customer_password,customer_street,customer_city,customer_state,customer_zipcode


In [5]:
def get_column_names(schemas: dict, ds_name: str, sorting_key='column_position'):
    """
    Get column names from schemas.json
    """
    column_details = schemas[ds_name]
    columns = sorted(column_details, key=lambda col: col[sorting_key], reverse=False)
    return [col['column_name'] for col in columns]

In [6]:
schemas = json.load(open('../data/retail_db/schemas.json'))
columns = get_column_names(schemas, 'orders')
df = pd.read_csv('../data/retail_db/orders/part-00000', names=columns)

In [12]:
df_reader = pd.read_csv('../data/retail_db/orders/part-00000', names=columns, chunksize=10000)

In [13]:
type(df_reader)

pandas.io.parsers.readers.TextFileReader

In [14]:
df_list = list(df_reader)
type(df_list)

list

In [17]:
# each item in df_list is a dataframe
type(df_list[0])

pandas.core.frame.DataFrame

In [22]:
for index, df in enumerate(df_list):
    print(f'Size of chunk {index} is {df.shape}')
    df.to_sql(
        name='orders',
        con=conn_uri,
        schema='retail',
        if_exists='append',
        index=False
    )

Size of chunk 0 is (10000, 4)
Size of chunk 1 is (10000, 4)
Size of chunk 2 is (10000, 4)
Size of chunk 3 is (10000, 4)
Size of chunk 4 is (10000, 4)
Size of chunk 5 is (10000, 4)
Size of chunk 6 is (8883, 4)


In [1]:
from dotenv import load_dotenv

In [2]:
# Load environment variables from the .env file
load_dotenv()

True

In [6]:
# Access the environment variables
os.getenv('SRC_BASE_DIR')
os.getenv('POSTGRES_HOST')

'de-essentials.c1i000w28fpm.ap-southeast-2.rds.amazonaws.com'